In [2]:
import openai
from openai import OpenAIError, OpenAI, AuthenticationError

In [3]:
API_KEY = "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"

In [7]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=API_KEY
)

In [25]:
input_text = input()

In [26]:
response = client.chat.completions.create(
                model="mistralai/mistral-7b-instruct:free",
                messages=[{
                    "role": "user",
                    "content": input_text,
                }]
            )

In [27]:
respuesta=response.choices[0].message.content

In [1]:
import spacy

# Carga el modelo de spaCy en español
nlp = spacy.load("es_core_news_md")

In [2]:
def remove_sensitive_data(text):
    """
    Elimina datos sensibles del texto (DNI, nombres, direcciones, etc.)
    """
    doc = nlp(text)
    sanitized_text = text

    # Detecta entidades y reemplaza o elimina las sensibles
    for ent in doc.ents:
        if ent.label_ in ["PER", "LOC", "ORG", "MISC"]:  # Nombres, lugares, organizaciones, etc.
            sanitized_text = sanitized_text.replace(ent.text, f"<{ent.label_}>")

        # Opcional: Puedes eliminar directamente la información en lugar de anonimizarla
        # sanitized_text = sanitized_text.replace(ent.text, "")

    return sanitized_text


In [3]:
test_text = "Juan Pérez vive en Madrid y su DNI es 12345678Z. Llámalo al 600123456."
cleaned_text = remove_sensitive_data(test_text)
print(cleaned_text)

<PER> vive en <LOC> y su DNI es 12345678Z. Llámalo al 600123456.


In [28]:
from IPython.display import Markdown, display
 
def print_markdown(text):
    """
    Prints a string in Markdown format in a Jupyter Notebook cell.
    Parameters:
        text (str): The string to display in Markdown format.
    """
    display(Markdown(text))

print_markdown(respuesta)

Aquí te presento un contrato de venta de finca en Madrid con un precio de 89.022,00 €. Es importante que los interesados en el contrato sean legalmente asesorados para asegurarse de que se entienden los términos y condiciones y que todo se realice de acuerdo con la legislación local y nacional aplicable.

**CONTRATO DE VENTA DE FINCA**

**PARTE CONTRATANTE:** [Nombre y apellidos del vendedor] (aquí después de leer y firmar)

**PARTE OBLIGADA:** [Nombre y apellidos del comprador] (aquí después de leer y firmar)

**ARTÍCULO 1 - OBLIGACIÓN DE VENTA**

1.1 El vendedor, por esta escritura, se compromete a vender a [nombre y apellidos del comprador] la finca situada en [dirección exacta de la finca] en la ciudad de Madrid, de acuerdo con las características descritas en el apartado siguiente.

**ARTÍCULO 2 - DESCRIPCIÓN DE LA FINCA**

2.1 La finca es una superficie de terreno urbano de 890,22 m², localizada en [dirección exacta de la finca] en la ciudad de Madrid.

2.2 La finca no presenta infracciones ni impedimentos legales, técnicos o de cualquier otro tipo que puedan impidir su libre disfrute.

**ARTÍCULO 3 - PRECIO**

3.1 El precio de la finca es de 89.022,00 €, incluido IVA y demás gastos que se deben cubrir por el vendedor.

**ARTÍCULO 4 - PAGO**

4.1 El comprador se compromete a pagar el precio total de la finca en una única ocasión al firmar esta escritura.

**ARTÍCULO 5 - ENTREGA DE LA FINCA**

5.1 El vendedor se compromete a entregar la finca en su estado actual al momento de la firma de esta escritura.

**ARTÍCULO 6 - DERECHOS REALES Y OTRAS INFRACCIONES**

6.1 El vendedor se compromete a presentar toda la documentación necesaria para garantizar la libre trasmisión de la finca al comprador.

**ARTÍCULO 7 - JURISDICCIÓN Y LEGISLACIÓN APLICABLE**

7.1 Todas las cuestiones relacionadas con este contrato se resolverán de acuerdo con la legislación española aplicable en materia de venta de bienes raíces.

**ARTÍCULO 8 - DISPOSIiciones ADICIONALES**

8.1 Todas las disposiciones adicionales que no se encuentren expresamente mencionadas en este contrato de venta de finca se regirán en todo caso por la legislación aplicable en materia de venta de bienes raíces.

**ARTÍCULO 9 - FIRMA Y VALIDEZ**

9.1 Para la validación de este contrato, es necesario que cada parte lo firme, lea y entienda su contenido antes de la firma.

Fecha: \_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_ de \_\_\_\_\_\_\_\_\_\_\_\_\_

(Vendedor) \_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_ (Comprador)

**NOTA:** Es importante que este contrato sea firmado ante notario público para que sea válido y que todo se realice de acuerdo con la legislación local y nacional aplicable.

**RECOMENDACIÓN:** El interesado debe consultar a un abogado especializado en materia de venta de bienes raíces antes de firmar este contrato para asegurarse de que se enti

In [11]:
import re
import spacy

nlp = spacy.load("es_core_news_md")

def replace_pattern(text, pattern, label, replacements):
    """
    Reemplaza un patrón específico en el texto y lo sustituye por una etiqueta anonimizada.
    """
    matches = re.findall(pattern, text)
    for match in matches:
        key = f"<{label}{len(replacements)}>"
        replacements[key] = match
        text = text.replace(match, key)
    return text

def anonymize_text(text):
    """
    Anonimiza datos sensibles como nombres, DNI, teléfonos y direcciones en el texto.
    """
    replacements = {}
    
    # Anonimizar nombres, organizaciones y ubicaciones con spaCy
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in ["PER", "LOC", "ORG"]:
            key = f"<{ent.label_}{len(replacements)}>"
            replacements[key] = ent.text
            text = text.replace(ent.text, key)

    # Anonimizar DNI (Formato español: 8 números + 1 letra)
    text = replace_pattern(text, r'\b\d{8}[A-Z]\b', "DNI", replacements)

    #  anonimizar cualquier número de teléfono (9+ dígitos seguidos)
    text = replace_pattern(text, r'\b\d{9,}\b', "TELEFONO", replacements)

    # Anonimizar direcciones basadas en palabras clave comunes
    residencia_patterns = [
        r"\b(vive en [\w\s,]+)",
        r"\b(reside en [\w\s,]+)",
        r"\b(con domicilio en [\w\s,]+)",
        r"\b(con residencia en [\w\s,]+)",
        r"\b(ubicado en [\w\s,]+)",
        r"\b(situado en [\w\s,]+)",
        r"\b(domiciliado en [\w\s,]+)",
        r"\b(se encuentra en [\w\s,]+)"
    ]

    for pattern in residencia_patterns:
        text = replace_pattern(text, pattern, "DIRECCION", replacements)

    return text, replacements

# **Prueba del código**
#test_text = "Juan Pérez vive en Madrid. Su número de casa es 912345678 y su móvil es 612345678. También tiene el DNI 12345678A."
#anonimized_text, replacements = anonymize_text(test_text)

#print("Texto anonimizado:", anonimized_text)
#print("Reemplazos:", replacements)


Texto anonimizado: <PER0> vive en <LOC1>. Su número de casa es <TELEFONO3> y su móvil es <TELEFONO4>. También tiene el DNI <DNI2>.
Reemplazos: {'<PER0>': 'Juan Pérez', '<LOC1>': 'Madrid', '<DNI2>': '12345678A', '<TELEFONO3>': '912345678', '<TELEFONO4>': '612345678'}


In [10]:
test_text = "Juan Pérez vive en Madrid. Tiene el DNI 12345678A y su teléfono es 612345678."
anonimized_text, replacements = anonymize_text(test_text)

print("Texto anonimizado:", anonimized_text)
print("Reemplazos:", replacements)


Texto anonimizado: <PER0> vive en <LOC1>. Tiene el DNI <DNI2> y su teléfono es <TELEFONO3>12345<TELEFONO3>78.
Reemplazos: {'<PER0>': 'Juan Pérez', '<LOC1>': 'Madrid', '<DNI2>': '12345678A', '<TELEFONO3>': '6'}


In [1]:
def validar_dni_espanol(dni):
    """
    Verifica si un DNI español es válido (formato y letra).
    """
    dni = dni.strip().upper()
    if not re.match(r'^\d{8}[A-Z]$', dni):
        return False

    letras = "TRWAGMYFPDXBNJZSQVHLCKE"
    numero = int(dni[:-1])
    letra_correcta = letras[numero % 23]

    return dni[-1] == letra_correcta


In [2]:
print(validar_dni_espanol("12345678Z"))  # ❌ False
print(validar_dni_espanol("12345678T"))  # ✅ True
print(validar_dni_espanol("87654321P"))  # ✅ True
print(validar_dni_espanol("11111111H"))  # ✅ True


NameError: name 're' is not defined

In [2]:
import openai

# Configuración de la API Key
with open(os.path.join(os.path.dirname(__file__), '..', 'secrets.json'), 'r', encoding='utf-8') as file:
    SECRETS = json.load(file)
# Configuración del cliente OpenAI
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=SECRETS["openai_api_key"],
)


response = client.chat.completions.create(
    model="mistralai/mistral-7b-instruct:free",
    messages=[{"role": "user", "content": "¿Qué es el usufructo?"}]
)

print(response)


NameError: name 'os' is not defined

In [4]:
import os
try:
    with open(os.path.join(os.path.dirname(__file__), '..', 'secrets.json'), 'r', encoding='utf-8') as file:
        SECRETS = json.load(file)
    print("🔐 Clave API cargada correctamente:", SECRETS.get("openai_api_key")[:5], "...")
except Exception as e:
    print("❌ Error cargando secrets.json:", e)


❌ Error cargando secrets.json: name '__file__' is not defined


In [5]:
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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"
)

response = client.chat.completions.create(
    model="mistralai/mistral-7b-instruct:free",
    messages=[{"role": "user", "content": "¿Qué es el usufructo?"}]
)

print(response)


ChatCompletion(id=None, choices=None, created=None, model=None, object=None, service_tier=None, system_fingerprint=None, usage=None, error={'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1745625600000'}, 'provider_name': None}}, user_id='user_2pm5szsJL4xO0cojp9nHbURtMja')
